# **Primer análisis de datos extraidos**


#### Incluimos las librerías necesarias para analizar los datos


In [2]:
import pandas as pd
import json
from collections import defaultdict, OrderedDict
import sorting
import re
from datetime import datetime, timedelta
import time
import xml.etree.ElementTree as ET
import keyword


#### Leemos los datos del JSON y lo guardamos en la variable _datos_


In [3]:
def myFunc(e):
    return e['timestamp']


datos = json.load(open('trazasOrdenadas.json'))


# Buscar si es nivel principal


In [4]:
def nivel_principal(nivel):
    erAnomalyLevels = re.compile(r'\btutorials|editor_level|tip|procedures')
    return not erAnomalyLevels.search(nivel)


# Obtener tarjetas de fin de nivel


In [5]:
def obtener_codigo(datos):
    # Recorre el array y guarda los bloques de tarjetas de los niveles completados
    # Las tarjetas se manipulan cuando:

    erLevel = re.compile(r'\blevel$\b')
    erButton = re.compile(r'button\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erProgressed = re.compile(r'\bprogressed$\b')
    erIdLevel = re.compile(r'/')
    tarjetas = defaultdict(defaultdict)
    intentos_fallidos = defaultdict(defaultdict)
    progresados = defaultdict(defaultdict)

    for evento in datos:
        obj = evento["object"]["definition"]["type"]
        objectId = evento["object"]["id"]
        verb = evento["verb"]["id"]
        name = evento["actor"]["name"]
        levelCode = erIdLevel.split(objectId)[-1]
        # Buscamos aquellos niveles que sean principales
        if nivel_principal(objectId):
            if erCompleted.search(verb):
                # Hemos encontrado un nivel completado de un codigo que teniamos guardado
                if levelCode in progresados and name in progresados[levelCode]:
                    if evento["result"]["success"] == True:
                        # Nivel completado con éxito
                        if name not in tarjetas[levelCode]:
                            tarjetas[levelCode][name] = [
                                progresados[levelCode][name]]
                        else:
                            tarjetas[levelCode][name].append(
                                progresados[levelCode][name])
                    else:
                        # Intento fallido
                        if name not in intentos_fallidos[levelCode]:
                            intentos_fallidos[levelCode][name] = [
                                progresados[levelCode][name]]
                        else:
                            intentos_fallidos[levelCode][name].append(
                                progresados[levelCode][name])

            elif "result" in evento:
                if "extensions" in evento["result"]:
                    if "code" in evento["result"]["extensions"]:
                        # Si hay codigo buscamos que el codigo sea final
                        if erCompleted.search(verb):
                            # Guarda el codigo de los intentos fallidos
                            if name not in intentos_fallidos[levelCode]:
                                intentos_fallidos[levelCode][name] = [
                                    evento["result"]["extensions"]["code"]]
                            else:
                                intentos_fallidos[levelCode][name].append(
                                    evento["result"]["extensions"]["code"])

                        elif erProgressed.search(verb):
                            # Comprobamos que todos los elementos que entran aquí son de codigo de niveles completados y no fallidos
                            progresados[levelCode][name] = evento["result"]["extensions"]["code"]

    return tarjetas, intentos_fallidos


codigo_final, codigo_fallido = obtener_codigo(datos)


## Obtenemos los tipos de tarjetas del codigo final


In [43]:
# Transforma el código en un árbol
    # La primera etiqueta es <xml>, contiene dos hijos <variables> y <block> respectivamente
	# <variables> contiene el nombr ey el id de las variables que se usarán en el código
	# <block> contiene una tarjeta, en este caso es start_start, que designa el punto de inicio
	# Y de manera recursiva cada hijo será la siguiente tarjeta que se ejecuta en el juego

def busquedaVariables(child):
	lVar = {}
	i = 1
	print ("--- Variables creadas ---")
	for variable in child:
		print("Variable "+ str(i))
		print("Nombre: "+ variable.text)
		#lvar[var1][0] es numModif de var1
		#lvar[var1][1] es numUsos de var1
		lVar[variable.text] = [0,0]
		i = i+1
	return lVar
	
def recorrerCodigo(start,varDict):
	settingVar = False
	gettingVar = False
	colaBucles = []
	looping = False
	varMult = 1
	tipoBloque = ""
	varName = ''

	#Vemos cuantas veces inicializa y usa cada variable
	for elem in start.iter():
		#Cambiamos el valor de una variable
		if elem.tag == "next":
			if looping:
				looping = False
				varMult = varMult / colaBucles[-1]
				colaBucles.pop()
		elif elem.tag == 'block':
			#Modificamos el valor de una variable
			if elem.attrib['type'] == "variables_set":
				settingVar = True
			elif elem.attrib['type'] == "variables_get":
				gettingVar = True
			elif elem.attrib['type'] == "controls_repeat":
				looping = True
		elif settingVar:
			if elem.attrib['name'] == "VAR":
				varName = elem.text
				varDict[varName][0] += varMult
				settingVar = False
		elif gettingVar:
			if elem.attrib['name'] == "VAR":
				varName = elem.text
				varDict[varName][1] += varMult 
				gettingVar = False
		elif looping:
			if elem.attrib['name'] == "TIMES":
				colaBucles.append(int(elem.text))
				varMult *= colaBucles[-1]

		
	
	return varDict

def nombreVarValido(nombresVar):
    pattern = re.compile(r'^_*[a-zA-Z][a-zA-Z0-9_]*$')
    declaracionVariables = []
    for nombre in nombresVar:
        if keyword.iskeyword(nombre):
            declaracionVariables.append(False)
        else:
            print(bool(pattern.match(nombre)))
            declaracionVariables.append(bool(pattern.match(nombre)))

    return declaracionVariables
	
def info_codigo(codigo):
	raiz = ET.fromstring(codigo)
	datos = defaultdict
	#Recorremos las variables principales
	antBloque = False
	for child in raiz:
		if child.tag == 'variables':
			listaVariables = busquedaVariables(child)
			#listaBooleanos = nombreVarValido(listaVariables)
		elif child.tag == 'block' and child.attrib['type'] == 'start_start':
			#Empezamos el código
			if antBloque:
				print("Codigo muerto")
			else:
				#Calcular uso de variables
				recorrerCodigo(child,listaVariables)
				antBloque = True
		else:
			antBloque = False
			
	return datos
    

def info_nivel(nivel):
    # Recorre el array y guarda información básica de cada nivel
    for usuario in nivel:
        # Convertimos el string a un árbol
        for codigo_str in nivel[usuario]:
            info_codigo(codigo_str)
			
info_nivel(codigo_final["loops_3"])

--- Variables creadas ---
Variable 1
Nombre: Objeto
Variable 2
Nombre: Desplazamiento
